# Model checking

In [6]:
import stormvogel.model
from stormvogel import show, model_checking, property_builder

In [7]:
mdp = stormvogel.model.new_mdp("Monty Hall")

init = mdp.get_initial_state()

# first choose car position
init.set_transitions(
    [(1 / 3, mdp.new_state("carchosen", {"car_pos": i})) for i in range(3)]
)

# we choose a door in each case
for s in mdp.get_states_with_label("carchosen"):
    s.set_transitions(
        [
            (
                mdp.action(f"open{i}"),
                mdp.new_state("open", s.features | {"chosen_pos": i}),
            )
            for i in range(3)
        ]
    )

# the other goat is revealed
for s in mdp.get_states_with_label("open"):
    car_pos = s.features["car_pos"]
    chosen_pos = s.features["chosen_pos"]
    other_pos = {0, 1, 2} - {car_pos, chosen_pos}
    s.set_transitions(
        [
            (
                1 / len(other_pos),
                mdp.new_state("goatrevealed", s.features | {"reveal_pos": i}),
            )
            for i in other_pos
        ]
    )

# we must choose whether we want to switch
for s in mdp.get_states_with_label("goatrevealed"):
    car_pos = s.features["car_pos"]
    chosen_pos = s.features["chosen_pos"]
    reveal_pos = s.features["reveal_pos"]
    other_pos = list({0, 1, 2} - {reveal_pos, chosen_pos})[0]
    s.set_transitions(
        [
            (
                mdp.action("stay"),
                mdp.new_state(
                    ["done"] + (["target"] if chosen_pos == car_pos else []),
                    s.features | {"chosen_pos": chosen_pos},
                ),
            ),
            (
                mdp.action("switch"),
                mdp.new_state(
                    ["done"] + (["target"] if other_pos == car_pos else []),
                    s.features | {"chosen_pos": other_pos},
                ),
            ),
        ]
    )

# we add self loops to all states with no outgoing transitions
mdp.add_self_loops()

show.show(mdp, show_editor=True)

Output()

Output()

We can do model checking on this model by only using stormvogel functions directly. Behind the scenes this calls the stormpy model checker. Before we use the model checker however, we need to specify the task. This is done using a property string. We have provided an easy way to create them for beginner users.

In [8]:
property_builder.build_property_string(mdp)

Output()

Output()

Create a property string and give it as argument to the model checking function in the following way

In [10]:
result = model_checking.model_checking(mdp) #true lets it return a scheduler as well

Output()

Output()

TypeError: parse_properties_without_context(): incompatible function arguments. The following argument types are supported:
    1. (formula_string: str, property_filter: Optional[Set[str]] = None) -> List[stormpy.core.Property]

Invoked with: None, None

We can visualize our model together with our results

In [31]:
show.show(mdp, result=result, show_editor=True)

Output()

Output()

To learn how to create more elaborate property strings, visit: https://www.stormchecker.org/documentation/background/properties.html